# Backsubstitution: a complete algorithm

IAM 961 Numerical Linear Algebra, University of New Hampshire, John Gibson, 2024--10-04

A complete example of a straightforward algorithm in Julia: backsubstitution, following 
Trefethen and Bau's wonderful book *Numerical Linear Algebra*.  The equation to solve is $Rx=\hat{b}$ where $R$ is square, upper triangular, and we'll assume of full rank, $\hat{b}$ is known, and $x$ is unknown. That system might come from solving an
$Ax=b$ problem with QR decomposition. That gives $A=QR$ with $Q$ unitary and $R$ upper-triangular. So $QRx=b$, and multiplying by $Q^*$ from the left gives $R x=Q^*b=\hat{b}$. 

### $3 \times 3$ case 

The $3 \times 3$ shows all the issues. I will drop the hats from the $b$s. 
\begin{align*} 
\begin{bmatrix} r_{11} & r_{12} & r_{13} \\ 0 & r_{22} & r_{23} \\  0 & 0 & r_{33} \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}
=
\begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}
\end{align*}

Proceeding up from the bottom row: 

The last row is $r_{33} \, x_3 = b_3$. If $A$ is full-rank, all diagonal elements $r_{jj}$ are non-zero,
so $x_3 = b_3/r_{33}$.

The second row is $r_{22} \, x_2 + r_{23} \, x_3 = b_2$. Since $x_3$ is now known, we solve for
$x_2 = (b_2 - r_{23} \, x_3)/r_{22}$.

The first row is $r_{11} \, x_1 + r_{12} \, x_2 + r_{13} \, x_3 = b_1$, giving 
$x_1 = (b_1 - r_{12} \, x_2 - r_{13} \, x_3)/r_{11}$.

### $m \times m$ generalization. 

The general solution algorithm is to iterate
\begin{align*}
x_i = \frac{1}{r_{ii}} \left(b_i - \sum_{j=i+1}^m r_{ij} \, x_j \right)
\end{align*}
starting at $i=m$ (the last row) and iterating up to $i=1$ (the top).

### Algorithm in Julia

In [1]:
# write and demo function in class, no frills, unoptimized, simple illustration 

"""
x = backsubstitution(R,b)

    solve upper-triangular system Rx=b for x using backsubstitution
    assume R is upper-tri, square, full rank
    teaching code, not for production use, no safety checks
"""
function backsubstitution(R,b)
    m,n = size(R)
    m!=n && error("R is not square. It is $m x $n")
    length(b) != m && error("b has wrong dimension, m = $m, but R is $m x $n")
    
    x = zeros(m)

    for i = m:-1:1
        s = b[i]
        for j = i+1:m
            s = s - R[i,j]*x[j]
        end
        x[i] = s/R[i,i]
    end

    x
end

backsubstitution

In [2]:
? backsubstitution

search: backsubstitution



x = backsubstitution(R,b)

```
solve upper-triangular system Rx=b for x using backsubstitution
assume R is upper-tri, square, full rank
teaching code, not for production use, no safety checks
```


### Test on random $Ax=b$ problem

In [3]:
# generate random Ax=b problem

A = randn(4,4)
x = randn(4)
b = A*x;

In [4]:
using LinearAlgebra

Q,R = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 4×4 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
4×4 Matrix{Float64}:
 -2.29205   0.405506   1.22387    0.602448
  0.0      -1.27463   -0.714049  -0.457083
  0.0       0.0        0.497101   1.21002
  0.0       0.0        0.0        2.04641

In [5]:
Q*R

4×4 Matrix{Float64}:
  0.0399739  -0.375149   0.191917  -0.0464277
 -1.60335     1.11096    1.36067    0.23009
  0.163207   -0.311361  -0.502972  -2.45537
  1.62927     0.563213  -0.337031  -0.373993

In [6]:
A

4×4 Matrix{Float64}:
  0.0399739  -0.375149   0.191917  -0.0464277
 -1.60335     1.11096    1.36067    0.23009
  0.163207   -0.311361  -0.502972  -2.45537
  1.62927     0.563213  -0.337031  -0.373993

In [7]:
x̂ = backsubstitution(R,Q'*b)

4-element Vector{Float64}:
  0.18038677335522893
  0.09730599070700964
  1.3776904191245558
 -0.24462429252719314

In [8]:
x

4-element Vector{Float64}:
  0.1803867733552291
  0.09730599070700963
  1.377690419124556
 -0.2446242925271933